In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9531901_2.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9768219_1.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9718992_2.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9138554_2.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9875303_1.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9720535_1.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9639683_2.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9857555_2.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9082631_2.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9133451_1.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9225063_1.png
/kaggle/input/knee-osteoarthritis-dataset-with-severity/auto_test/2/9525665_2.png
/kaggle/input/kn

In [2]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import v2
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")
import random
import shutil

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [3]:
clahe = cv2.createCLAHE(clipLimit=0.03, tileGridSize=(8, 8))

In [4]:
train_dict = {}
val_dict = {}
test_dict = {}
train_path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity/train/"
val_path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity/val/"
test_path = "/kaggle/input/knee-osteoarthritis-dataset-with-severity/test/"

In [5]:
for label in os.listdir(train_path):
    train_dict[int(label)] = os.listdir(train_path + label)
for label in os.listdir(test_path):
    test_dict[int(label)] = os.listdir(test_path + label)
for label in os.listdir(val_path):
    val_dict[int(label)] = os.listdir(val_path + label)

In [6]:
train_images = {}
val_images = {}
test_images = {}

In [7]:
for keys in train_dict.keys():  
    train_normal_images = [np.array(Image.open(train_path + str(keys) + "/" + images)) for images in train_dict[keys]]
    test_normal_images = [np.array(Image.open(test_path + str(keys) + "/" + images)) for images in test_dict[keys]]
    val_normal_images = [np.array(Image.open(val_path + str(keys) + "/" + images)) for images in val_dict[keys]]
    train_images[keys] = train_normal_images
    test_images[keys] = test_normal_images
    val_images[keys] = val_normal_images

In [8]:
trlabel0 = [0 for i in range(len(train_images[0]))]
trlabel1 = [1 for i in range(len(train_images[1]))]
trlabel2 = [2 for i in range(len(train_images[2]))]
trlabel3 = [3 for i in range(len(train_images[3]))]
trlabel4 = [4 for i in range(len(train_images[4]))]

In [9]:
tslabel0 = [0 for i in range(len(test_images[0]))]
tslabel1 = [1 for i in range(len(test_images[1]))]
tslabel2 = [2 for i in range(len(test_images[2]))]
tslabel3 = [3 for i in range(len(test_images[3]))]
tslabel4 = [4 for i in range(len(test_images[4]))]

In [10]:
vlabel0 = [0 for i in range(len(val_images[0]))]
vlabel1 = [1 for i in range(len(val_images[1]))]
vlabel2 = [2 for i in range(len(val_images[2]))]
vlabel3 = [3 for i in range(len(val_images[3]))]
vlabel4 = [4 for i in range(len(val_images[4]))]

In [11]:
training_image  = torch.tensor(torch.cat((torch.tensor(train_images[0]),torch.tensor(train_images[1]),torch.tensor(train_images[2]),torch.tensor(train_images[3]),torch.tensor(train_images[4])),0),dtype = torch.float32)
training_labels = torch.tensor(torch.cat((torch.tensor(trlabel0),torch.tensor(trlabel1),torch.tensor(trlabel2),torch.tensor(trlabel3),torch.tensor(trlabel4)),0))


In [12]:
training_image = training_image.view(training_image.shape[0],1,224,224)

In [13]:
testing_image  = torch.tensor(torch.cat((torch.tensor(test_images[0]),torch.tensor(test_images[1]),torch.tensor(test_images[2]),torch.tensor(test_images[3]),torch.tensor(test_images[4])),0),dtype = torch.float32)
testing_labels = torch.tensor(torch.cat((torch.tensor(tslabel0),torch.tensor(tslabel1),torch.tensor(tslabel2),torch.tensor(tslabel3),torch.tensor(tslabel4)),0))


In [14]:
testing_image = testing_image.view(testing_image.shape[0],1,224,224)

In [15]:
val_image  = torch.tensor(torch.cat((torch.tensor(val_images[0]),torch.tensor(val_images[1]),torch.tensor(val_images[2]),torch.tensor(val_images[3]),torch.tensor(val_images[4])),0),dtype = torch.float32)
val_labels = torch.tensor(torch.cat((torch.tensor(vlabel0),torch.tensor(vlabel1),torch.tensor(vlabel2),torch.tensor(vlabel3),torch.tensor(vlabel4)),0))


In [16]:
val_image = val_image.view(val_image.shape[0],1,224,224)

In [17]:
train_dataset = TensorDataset(training_image,training_labels)
val_dataset = TensorDataset(val_image,val_labels)
test_dataset = TensorDataset(testing_image,testing_labels)

In [58]:
batch_size = 4

train_loader = DataLoader(train_dataset,shuffle= True , batch_size = batch_size)
val_loader = DataLoader(val_dataset,shuffle = True , batch_size = batch_size)
test_loader = DataLoader(test_dataset,shuffle = True, batch_size = batch_size)

In [51]:
class NewModel(nn.Module):
    def __init__(self,model):
        super(NewModel,self).__init__()
        features = list(model.features.children())[1:-2]
        for i in range(len(features)):
            if isinstance(features[i], nn.Conv2d):
                features.insert(i+1, nn.BatchNorm2d(features[i].out_channels))
                
        self.features = nn.Sequential(
                        nn.Conv2d(1,64,kernel_size = 3,padding = 1,stride = 1),
                        *features,
                        nn.ReLU(inplace = True),
                        nn.MaxPool2d(2))
        
        self.avgpool = nn.AvgPool2d(2)
        
        self.classifier = nn.Sequential(
                            nn.Linear(512*3*3,512),
                            nn.BatchNorm1d(512),
                            nn.ReLU(inplace = True),
                            nn.Dropout(p = 0.2,inplace = False),
                            nn.Linear(512,5),
                            )
        
            
    def forward(self,x):
            x = self.features(x)
            x = self.avgpool(x)

            x = x.view(x.shape[0],-1)
            
            x = self.classifier(x)
            
            return x

In [52]:
original_model = torchvision.models.vgg16(pretrained = True)

In [53]:
new_model = NewModel(original_model)

In [54]:
new_model.eval()

NewModel(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(256, eps=1

In [55]:
filtered_state_dict = {k: v for k, v in original_model.state_dict().items() if k in new_model.state_dict() and v.shape == new_model.state_dict()[k].shape}

In [56]:
new_model.load_state_dict(filtered_state_dict, strict=False)

_IncompatibleKeys(missing_keys=['features.0.weight', 'features.3.weight', 'features.3.bias', 'features.3.running_mean', 'features.3.running_var', 'features.6.weight', 'features.6.bias', 'features.7.weight', 'features.7.running_mean', 'features.7.running_var', 'features.9.weight', 'features.9.bias', 'features.10.weight', 'features.10.bias', 'features.10.running_mean', 'features.10.running_var', 'features.13.weight', 'features.13.bias', 'features.14.weight', 'features.14.running_mean', 'features.14.running_var', 'features.16.weight', 'features.16.bias', 'features.17.weight', 'features.17.bias', 'features.17.running_mean', 'features.17.running_var', 'features.19.weight', 'features.19.bias', 'features.20.weight', 'features.20.bias', 'features.20.running_mean', 'features.20.running_var', 'features.23.weight', 'features.23.bias', 'features.24.weight', 'features.24.running_mean', 'features.24.running_var', 'features.27.weight', 'features.27.bias', 'features.27.running_mean', 'features.27.runn

In [57]:
no_of_layers = 22

for i, params in enumerate(new_model.parameters()):
    if i < no_of_layers:
        params.requires_grad = False
        
# for i, params in enumerate(new_model.parameters()):
#     if i >= 25:
#         if len(params.shape) == 1:
#             params.data = torch.zeros(params.shape[0])
#         else:
#             params.data = torch.zeros(params.shape[0],params.shape[1])


In [59]:
device = 'cuda'

new_model.to(device)

NewModel(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(256, eps=1

In [60]:
criterion = nn.CrossEntropyLoss()

In [61]:
optimizer = torch.optim.AdamW(new_model.parameters(),lr = 0.0001)

In [66]:
training_loss = 0.0
training_accuracy = 0
validation_accuracy = 0

for epoch in range(15):
    n_samples_train = 0
    n_samples_val = 0
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        output = new_model(images)
        loss = criterion(output, labels)
        n_samples_train += labels.size(0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(output.data, 1)
        training_accuracy += (predicted == labels).sum().item()

        if (i + 1) % 64 == 0:
            print(f'Training - Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {loss.item():.4f}')

    print(f'Training Accuracy - Epoch: {epoch + 1}: {training_accuracy * 100 / n_samples_train:.2f}%')
    training_accuracy = 0

    with torch.no_grad():
        for i, (val_images, val_labels) in enumerate(val_loader):

            val_images = val_images.to(device)
            val_labels = val_labels.to(device)
            val_output = new_model(val_images)
            _, val_predicted = torch.max(val_output.data, 1)
            validation_accuracy += (val_predicted == val_labels).sum().item()
            n_samples_val += val_labels.size(0)

    print(f'Validation Accuracy - Epoch: {epoch + 1}: {validation_accuracy * 100 / n_samples_val:.2f}%')
    validation_accuracy = 0


Training - Epoch: 1, Batch: 64, Loss: 0.7636
Training - Epoch: 1, Batch: 128, Loss: 0.4919
Training - Epoch: 1, Batch: 192, Loss: 1.6813
Training - Epoch: 1, Batch: 256, Loss: 0.3298
Training - Epoch: 1, Batch: 320, Loss: 0.3087
Training - Epoch: 1, Batch: 384, Loss: 2.0226
Training - Epoch: 1, Batch: 448, Loss: 0.6867
Training - Epoch: 1, Batch: 512, Loss: 0.0667
Training - Epoch: 1, Batch: 576, Loss: 0.4937
Training - Epoch: 1, Batch: 640, Loss: 0.5526
Training - Epoch: 1, Batch: 704, Loss: 1.1705
Training - Epoch: 1, Batch: 768, Loss: 0.9117
Training - Epoch: 1, Batch: 832, Loss: 0.6400
Training - Epoch: 1, Batch: 896, Loss: 0.1581
Training - Epoch: 1, Batch: 960, Loss: 0.9150
Training - Epoch: 1, Batch: 1024, Loss: 0.2281
Training - Epoch: 1, Batch: 1088, Loss: 0.5906
Training - Epoch: 1, Batch: 1152, Loss: 1.7832
Training - Epoch: 1, Batch: 1216, Loss: 0.1702
Training - Epoch: 1, Batch: 1280, Loss: 0.0857
Training - Epoch: 1, Batch: 1344, Loss: 1.9593
Training - Epoch: 1, Batch: 1

In [67]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    test_loss = 0.0
    test_accuracy = 0
    for i,(images, labels) in enumerate(test_loader):

        images = images.to(device)
        labels = labels.to(device)
        outputs = new_model(images)
        loss = criterion(outputs.data,labels)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        test_accuracy = n_correct
        test_loss = loss.item()
        if((i+1)%50 == 0):

            test_accuracy = 0
            test_loss = 0
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images:{acc:.4f}%')

Accuracy of the network on the 1656 test images:53.2005%
